In [5]:
import pandas as pd
# removed this from wiglecsv: WigleWifi-1.4,appRelease=Kismet202208R1,model=Kismet,release=2022.08.R1,device=kismet,display=kismet,board=kismet,brand=kismet
csvd = pd.read_csv('/home/vlaghe/proj/warpi/data/buc2run/buc2runweli-20230114-14-53-28-1.wiglecsv')
# csvd = pd.read_csv('/path/to/your/file.wiglecsv')
#csvd.describe()  # summarry of data
#csvd.AuthMode.unique() # get uniq vals 4 one column
#csvd.drop_duplicates(subset='MAC') # drop duplicates based on col
csvd.sample(50)  # show 10 random sameples

,MAC,SSID,AuthMode,FirstSeen,Channel,RSSI,CurrentLatitude,CurrentLongitude,AltitudeMeters,AccuracyMeters,Type
32369,C4:6E:1F:0B:D8:24,PEVAL,[WPA2-PSK-CCMP+TKIP][ESS],2023-01-14 16:50:00,6,-77,44.454887,26.110613,88.8,0,WIFI
16857,F7:68:07:8A:9A:3A,F7:68:07:8A:9A:3A,Misc [LE],2023-01-14 15:24:48,0,0,44.434550,26.077987,63.6,0,BLE
24738,BE:7A:BF:88:3D:8C,AndroidAP1757,[WPA2-PSK-CCMP][ESS],2023-01-14 14:53:58,6,-59,44.428680,26.053144,100.6,0,WIFI
22845,41:E1:F0:F4:18:C5,41:E1:F0:F4:18:C5,Misc [LE],2023-01-14 16:23:05,0,0,44.428917,26.053610,91.6,0,BLE
9851,40:EE:DD:F1:76:B4,DIGI-sJ2V,[WPA-PSK-CCMP] [WPA2-PSK-CCMP][ESS],2023-01-14 15:10:46,2,-77,44.436835,26.101877,77.4,0,WIFI
32662,5A:03:04:2E:01:D5,5A:03:04:2E:01:D5,Misc [LE],2023-01-14 16:51:27,0,0,44.453225,26.111200,77.8,0,BLE
4710,DA:0D:17:2C:D8:C0,BEJDI_Guest,[WPA2-PSK-CCMP+TKIP][ESS],2023-01-14 15:03:14,11,-71,44.443801,26.119680,119.8,0,WIFI
4954,AC:22:05:E0:A8:D6,Vodafone-D6C9,[WPA2-PSK-CCMP][ESS],2023-01-14 15:03:22,1,-71,44.443771,26.119670,116.9,0,WIFI
26342,E9:51:A2:53:95:16,E9:51:A2:53:95:16,Misc [LE],2023-01-14 16:25:12,0,0,44.429240,26.052320,136.7,0,BLE
20220,D4:9D:C0:52:A1:31,D4:9D:C0:52:A1:31,Misc [LE],2023-01-14 15:34:58,0,0,44.428743,26.051932,86.7,0,BLE


## 📊 Some Stats 
Simple statistics about our data:
- Most Used SSIDs
- Open VS Hidden SSIDs
- Frequently Used Channels
- WiFi Security Protocols *(Open, WPA[1-3], Mixed)*

### Most Used SSIDs
I wanted to take a look @ the most common names when it comes to choosing SSIDs

From the perspective of an attacker / WiFi assessment this data can be quite fruitful, given that you can map the most used SSIDs in an area & use that to lunch further attacks for initail access *(e.g rogue AP, phising)* 

> **NOTE**: this is calculated from all of my dataset => the frequency @ which SSID names are counted are directly correlated with the GPS coords => if I stayed more in 1 place, the SSIDs in that area will show more times than others 

In [6]:
# Filter out MAC addresses from SSIDs
#   && get counts of unique names
regex = r"^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$" 
SSIDs = csvd[csvd.SSID.str.match(regex)==False].SSID
counts = SSIDs.value_counts()

# drop the first 2 columns (my phone hotspot & car camera)
# counts = counts.iloc[2:]    
# counts.head(10) # | counts.index

TOP = 25    # change me to your liking 

# Style plot
plt.style.use('dark_background')
counts[:TOP].plot(kind='barh', color='darkcyan')
sns.set(font_scale=0.8)
plt.ylabel("SSID", labelpad=14)
plt.xlabel("Count of rcvd beacons", labelpad=14)
plt.title("Top 20 most used SSIDs", y=1.02);

# Want sorting?
# sort alphabetically : .sort_index(ascending=True)
# sort by value       : .sort_values(ascending=False)
# get % count         : .value_counts(normalize=True)

# total unique SSIDs: csvd.SSID.nunique()
# uniq_ssids = csvd.SSID.unique()

NameError: name 'plt' is not defined